In [22]:
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
import tetrion.plots as plots
import tetrion.db
import tetrion.commands as cmd
import tetrion.options
import tetrion.clientcore
import seaborn as sns
from collections import defaultdict
import numpy as np
import warnings
import matplotlib.pyplot as plt
warnings.simplefilter("ignore")
%matplotlib inline 
symbol = pd.read_csv('/local/dist/tetrion/conf/conf/symbol.table', skiprows= 1, header = 2,  delim_whitespace=True)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
# import sys
# sys.path.append('../utils')  # Adjust the path to where the eventstudy module is located
# import eventstudy as es
import pandas as pd
import warnings
from tqdm import tqdm

In [23]:
warnings.filterwarnings("ignore", category=DeprecationWarning)
start_date = '2025-06-02'
end_date = '2025-06-09'
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

txf_books, txf_ticks = pd.DataFrame(), pd.DataFrame()

for date in tqdm(date_range, desc="Loading TXF data by day"):
    try:
        txf_book = cmd.book_printer_v2(
            'FUT_TAIFEX_TXF:202507',
            sdate=date,
            edate=date,
            depth=5,
            source='SRC_TAIFEX_RT_RD.ORIG_CPBQ.DEST_CPBQ'
        )
        txf_tick = cmd.tick_printer(
            'FUT_TAIFEX_TXF:202507',
            sdate=date,
            edate=date
        )

        txf_books = pd.concat([txf_books, txf_book], axis=0)
        txf_ticks = pd.concat([txf_ticks, txf_tick], axis=0)

    except Exception:
        continue


Loading TXF data by day:  75%|███████▌  | 6/8 [01:43<00:28, 14.23s/it]WARNING: MYSQL_OPT_RECONNECT is deprecated and will be removed in a future version.
missing datafile SRC_TAIFEX.ORIG_CPBQ.DEST_CPBQ FUT_TAIFEX_TXF 20250608 file=/nfs/datafiles/2025/06/08/59709/datafile.944111726.gz
Loading TXF data by day: 100%|██████████| 8/8 [01:44<00:00, 13.02s/it]


In [24]:
ticks_for_analysis = txf_ticks.copy()
ticks_for_analysis['ex_time'] = pd.to_datetime(ticks_for_analysis['ex_time'])
ticks_for_analysis = ticks_for_analysis.rename(columns={'ex_time': 'time'}).set_index('time')


In [25]:
merged_df = pd.merge(ticks_for_analysis[['sz','px']], txf_books, on='time', how='outer', sort=True)
merged_df['sz'] = merged_df['sz'].fillna(0)

merged_df[txf_books.columns] = merged_df[txf_books.columns].ffill()

In [26]:
traded = merged_df[merged_df['sz'] != 0].copy()

# 定義穿價條件（避免 ask_px4 或 bid_px4 為 0）
up_cross = (traded['ask_px2'] != 0) & (traded['px'] > traded['ask_px2'])
down_cross = (traded['bid_px2'] != 0) & (traded['px'] < traded['bid_px2'])

# 建立穿價標籤
traded['trade_through'] = 'none'
traded.loc[up_cross, 'trade_through'] = 'upward'
traded.loc[down_cross, 'trade_through'] = 'downward'


# 只留下有穿價的成交
trade_through_df = traded[traded['trade_through'] != 'none']

In [45]:
txf_books.loc["2025-06-07"].between_time("00:45", "04:00")

,bid_sz4,bid_sz3,bid_sz2,bid_sz1,bid_sz,bid_px4,bid_px3,bid_px2,bid_px1,bid_px,mid_px,ask_px,ask_px1,ask_px2,ask_px3,ask_px4,ask_sz,ask_sz1,ask_sz2,ask_sz3,ask_sz4
time,,,,,,,,,,,,,,,,,,,,,
2025-06-07 00:45:00.106549,3.0,1.0,1.0,2.0,8,21350.0,21353.0,21360.0,21364.0,21365.0,21367.5,21370.0,21372.0,21373.0,21374.0,21412.0,3,1.0,1.0,1.0,1.0
2025-06-07 00:45:00.106553,3.0,1.0,1.0,2.0,8,21350.0,21353.0,21360.0,21364.0,21365.0,21367.5,21370.0,21372.0,21373.0,21412.0,21415.0,3,1.0,1.0,1.0,2.0
2025-06-07 00:45:00.106897,1.0,1.0,1.0,2.0,8,21353.0,21360.0,21361.0,21364.0,21365.0,21367.5,21370.0,21372.0,21373.0,21412.0,21415.0,3,1.0,1.0,1.0,2.0
2025-06-07 00:45:00.106934,1.0,1.0,1.0,2.0,8,21353.0,21360.0,21361.0,21364.0,21365.0,21367.5,21370.0,21372.0,21373.0,21411.0,21412.0,3,1.0,1.0,1.0,1.0
2025-06-07 00:45:00.106948,1.0,1.0,1.0,2.0,8,21353.0,21360.0,21361.0,21364.0,21365.0,21367.5,21370.0,21372.0,21373.0,21374.0,21411.0,3,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-07 03:59:59.894348,1.0,2.0,1.0,3.0,1,21340.0,21350.0,21352.0,21367.0,21368.0,21370.5,21373.0,21377.0,21379.0,21380.0,21381.0,2,1.0,1.0,1.0,1.0
2025-06-07 03:59:59.894348,1.0,2.0,1.0,3.0,1,21340.0,21350.0,21352.0,21367.0,21368.0,21370.5,21373.0,21377.0,21379.0,21380.0,21381.0,1,1.0,1.0,1.0,1.0
2025-06-07 03:59:59.894348,1.0,2.0,1.0,2.0,1,21340.0,21350.0,21352.0,21367.0,21368.0,21370.5,21373.0,21377.0,21379.0,21380.0,21381.0,1,1.0,1.0,1.0,1.0


In [31]:
trade_through_df


,sz,px,bid_sz4,bid_sz3,bid_sz2,bid_sz1,bid_sz,bid_px4,bid_px3,bid_px2,bid_px1,bid_px,mid_px,ask_px,ask_px1,ask_px2,ask_px3,ask_px4,ask_sz,ask_sz1,ask_sz2,ask_sz3,ask_sz4,trade_through
time,,,,,,,,,,,,,,,,,,,,,,,,
2025-06-03 00:00:55.798,1.0,20693.0,1.0,1.0,1.0,2.0,2.0,20661.0,20693.0,20694.0,20698.0,20699.0,20700.5,20702.0,20703.0,20704.0,20706.0,20713.0,5.0,1.0,1.0,1.0,1.0,downward
2025-06-03 00:26:32.437,1.0,20687.0,1.0,1.0,1.0,2.0,2.0,20661.0,20693.0,20694.0,20698.0,20699.0,20700.5,20702.0,20703.0,20704.0,20706.0,20713.0,5.0,1.0,1.0,1.0,1.0,downward
2025-06-03 00:26:32.437,1.0,20687.0,1.0,1.0,1.0,2.0,2.0,20661.0,20693.0,20694.0,20698.0,20699.0,20700.5,20702.0,20703.0,20704.0,20706.0,20713.0,5.0,1.0,1.0,1.0,1.0,downward
2025-06-03 00:55:10.294,1.0,20712.0,1.0,1.0,1.0,2.0,2.0,20661.0,20693.0,20694.0,20698.0,20699.0,20700.5,20702.0,20703.0,20704.0,20706.0,20713.0,5.0,1.0,1.0,1.0,1.0,upward
2025-06-03 00:56:31.159,1.0,20712.0,1.0,1.0,1.0,2.0,2.0,20661.0,20693.0,20694.0,20698.0,20699.0,20700.5,20702.0,20703.0,20704.0,20706.0,20713.0,5.0,1.0,1.0,1.0,1.0,upward
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-09 13:44:00.702,1.0,21425.0,1.0,1.0,2.0,2.0,2.0,21334.0,21340.0,21350.0,21352.0,21362.0,21367.0,21372.0,21398.0,21400.0,21410.0,21412.0,1.0,1.0,2.0,1.0,1.0,upward
2025-06-09 13:44:06.019,1.0,21430.0,1.0,1.0,2.0,2.0,2.0,21334.0,21340.0,21350.0,21352.0,21362.0,21367.0,21372.0,21398.0,21400.0,21410.0,21412.0,1.0,1.0,2.0,1.0,1.0,upward
2025-06-09 13:44:09.388,1.0,21431.0,1.0,1.0,2.0,2.0,2.0,21334.0,21340.0,21350.0,21352.0,21362.0,21367.0,21372.0,21398.0,21400.0,21410.0,21412.0,1.0,1.0,2.0,1.0,1.0,upward


In [49]:
aaa = cmd.book_printer_v2('FUT_TAIFEX_TXF:202506',sdate=dt.date(2025,6,5),depth=5,source='SRC_TAIFEX_RT_RD.ORIG_CPBQ.DEST_CPBQ')

In [ ]:
aaa.loc["2025-06-05"].between_time("00:45", "04:00")

KeyError: '2025-06-07'

In [13]:
import sys
import os
sys.path.append(os.path.abspath(".."))
from eddie_tools.record import RedisFillLoader

In [14]:
tmf = RedisFillLoader('2025,4,25','2025,5,8','capital_electron_tmf')

In [15]:
df = tmf.load_concat_df()
fills_df = df[df['instr'].str.contains('tmf:202505', case=False, na=False)]

Concatenating fills: 100%|██████████| 14/14 [00:01<00:00, 10.93it/s]


In [16]:
def match_fills_to_trade_through(fills_df, trade_through_df, threshold_seconds=0.1):
    import pandas as pd

    fills_df = fills_df.copy()
    trade_through_df = trade_through_df.copy()

    # 確保 time 是欄位，不是 index
    if fills_df.index.name == 'time' or 'time' not in fills_df.columns:
        fills_df = fills_df.reset_index()

    if trade_through_df.index.name == 'time' or 'time' not in trade_through_df.columns:
        trade_through_df = trade_through_df.reset_index()

    fills_df = fills_df.sort_values("time")
    trade_through_df = trade_through_df.sort_values("time")

    if 'trade_through' not in trade_through_df.columns:
        raise ValueError("`trade_through_df` must contain 'trade_through' column")

    trade_through_df = trade_through_df[['time', 'trade_through']]

    # merge_asof 按時間對應最近穿價事件
    merged = pd.merge_asof(
        fills_df,
        trade_through_df,
        on='time',
        direction='backward',
        tolerance=pd.Timedelta(seconds=threshold_seconds)
    )

    merged["is_trade_near_cross"] = merged["trade_through"].notna()
    return merged


In [17]:
def analyze_ratio_count_and_volume(df):
    df = df.copy()
    df['trade_through'] = df['trade_through'].fillna('no-through')

    for label, sub_df in [('long', df[df['sz'] > 0]), ('short', df[df['sz'] < 0])]:
        total_count = len(sub_df)
        total_volume = sub_df['sz'].abs().sum()

        if total_count == 0 or total_volume == 0:
            print(f"\n沒有 {label.upper()} TMF")
            continue

        ratio_count = sub_df['trade_through'].value_counts(normalize=True)
        ratio_volume = sub_df.groupby('trade_through')['sz'].apply(lambda x: x.abs().sum()) / total_volume

        print(f"\nElectron {label.upper()} 202505TMF 時 :")
        print(f"使用成交口數加權-> TXF 價格上穿: {ratio_volume.get('upward', 0):.2%}, TXF 價格下穿: {ratio_volume.get('downward', 0):.2%}, 無穿價: {ratio_volume.get('no-through', 0):.2%}")


In [18]:
merged_fills = match_fills_to_trade_through(fills_df, trade_through_df, threshold_seconds=0.01)

In [19]:
analyze_ratio_count_and_volume(merged_fills)


Electron LONG 202505TMF 時 :
使用成交口數加權-> TXF 價格上穿: 19.23%, TXF 價格下穿: 8.16%, 無穿價: 72.61%

Electron SHORT 202505TMF 時 :
使用成交口數加權-> TXF 價格上穿: 10.23%, TXF 價格下穿: 37.04%, 無穿價: 52.74%


In [20]:
merged_fills_night = merged_fills.set_index('time').between_time('15:00:00','05:00:00')

In [21]:
analyze_ratio_count_and_volume(merged_fills_night)


Electron LONG 202505TMF 時 :
使用成交口數加權-> TXF 價格上穿: 16.48%, TXF 價格下穿: 8.70%, 無穿價: 74.82%

Electron SHORT 202505TMF 時 :
使用成交口數加權-> TXF 價格上穿: 11.78%, TXF 價格下穿: 40.80%, 無穿價: 47.42%


* 使用加權的原因：原本計算方式不太合理 會使short 1口 TMF以及 short 20口 tmf擁有相同比重 因此使用sz加權算機率較合邏輯

In [15]:
merged_fills_night

,ts,instr,oflag,tid,px,alqortake,sz,theo,fwd,ffwd,edge,roll_offset,label,delta,vega,gamma,theta,vol,mult,latefill,fee,qm,pfdelta,otags,seq,trade_through,is_trade_near_cross
time,,,,,,,,,,,,,,,,,,,,,,,,,,,
2025-04-24 19:39:07.756836,1745494747756836,FUT_TAIFEX_TMF:202505,0,TAIFEX155#290618,19520,alq,1,19521.0,19521.0,19521,0,0,,1,0,0,0,0,10,False,1.19040,0,0,0x20000,23,NaN,False
2025-04-24 19:39:07.938710,1745494747938710,FUT_TAIFEX_TMF:202505,0,TAIFEX155#290659,19523,take,20,19522.5,19522.5,19522.5,0,0,,1,0,0,0,0,10,True,1.19046,0,0,0x20000,24,NaN,False
2025-04-24 19:39:07.939160,1745494747939160,FUT_TAIFEX_TMF:202505,0,TAIFEX155#290661,19524,take,5,19524.5,19524.5,19524.5,0,0,,1,0,0,0,0,10,False,1.19048,0,0,0x0,26,NaN,False
2025-04-24 19:39:07.939194,1745494747939194,FUT_TAIFEX_TMF:202505,0,TAIFEX155#290661,19525,take,15,19524.5,19524.5,19524.5,0,0,,1,0,0,0,0,10,False,1.19050,0,0,0x0,27,NaN,False
2025-04-24 19:39:07.946233,1745494747946233,FUT_TAIFEX_TMF:202505,0,TAIFEX155#290668,19525,alq,-1,19524.5,19524.5,19524.5,0,0,,1,0,0,0,0,10,False,1.19050,0,0,0x20000,29,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-08 04:59:33.838739,1746651573838739,FUT_TAIFEX_TMF:202505,0,TAIFEX155#997839,20519,take,-1,20518.5,20518.5,20518.5,0,0,,1,0,0,0,0,10,False,1.21038,0,0,0x0,262,NaN,False
2025-05-08 04:59:33.838796,1746651573838796,FUT_TAIFEX_TMF:202505,0,TAIFEX155#997839,20515,take,-5,0.0,0.0,NaN,0,0,,1,0,0,0,0,10,False,1.21030,0,0,0x0,266,NaN,False
2025-05-08 04:59:33.838796,1746651573838796,FUT_TAIFEX_TMF:202505,0,TAIFEX155#997839,20516,take,-7,20515.5,20515.5,20515.5,0,0,,1,0,0,0,0,10,False,1.21032,0,0,0x0,265,NaN,False


In [22]:
import numpy as np

def analyze_ratio_count_and_volume2(df):
    df = df.copy()
    df['trade_through'] = df['trade_through'].fillna('no-through')

    for label, sub_df in [('long', df[df['sz'] > 0]), ('short', df[df['sz'] < 0])]:
        if sub_df.empty:
            print(f"\n沒有 {label.upper()} TMF")
            continue

        # 成交量加權機率
        weights = sub_df['sz'].abs()

        # 先轉成 one-hot 類型欄位
        upward = (sub_df['trade_through'] == 'upward').astype(int)
        downward = (sub_df['trade_through'] == 'downward').astype(int)
        no_through = (sub_df['trade_through'] == 'no-through').astype(int)

        # 加權平均
        weighted_upward = np.average(upward, weights=weights)
        weighted_downward = np.average(downward, weights=weights)
        weighted_no_through = np.average(no_through, weights=weights)

        
        ratio_count = sub_df['trade_through'].value_counts(normalize=True)

        print(f"\nElectron {label.upper()} 202505TMF 時:")
        print(f"只看成交次數比率 -> 上穿: {ratio_count.get('upward', 0):.2%}, 下穿: {ratio_count.get('downward', 0):.2%}, 無穿價: {ratio_count.get('no-through', 0):.2%}")
        print(f"成交口數加權機率 -> 上穿: {weighted_upward:.2%}, 下穿: {weighted_downward:.2%}, 無穿價: {weighted_no_through:.2%}")


In [23]:
analyze_ratio_count_and_volume2(merged_fills_night)


Electron LONG 202505TMF 時:
只看成交次數比率 -> 上穿: 10.28%, 下穿: 5.87%, 無穿價: 83.85%
成交口數加權機率 -> 上穿: 16.48%, 下穿: 8.70%, 無穿價: 74.82%

Electron SHORT 202505TMF 時:
只看成交次數比率 -> 上穿: 4.95%, 下穿: 26.50%, 無穿價: 68.55%
成交口數加權機率 -> 上穿: 11.78%, 下穿: 40.80%, 無穿價: 47.42%


* old version group by second

In [ ]:
# def group_fills_within_seconds(fills_df, threshold_seconds=1):
#     # df = fills_df.reset_index().sort_values('time').copy()  # <-- 這裡加上 reset_index()
#     df['time'] = pd.to_datetime(df['time'])
#     df['time_diff'] = df['time'].diff().dt.total_seconds().fillna(float('inf'))
#     df['group_id'] = (df['time_diff'] > threshold_seconds).cumsum()

#     grouped = (
#         df.groupby('group_id')
#         .agg(
#             time=('time', 'first'),
#             sz=('sz', 'sum'),
#             px=('px', 'mean'),
#             instr=('instr', 'first')
#         )
#         .reset_index(drop=True)
#     )

#     return grouped


In [ ]:
# def match_grouped_fills_to_trade_through(grouped_fills_df, trade_through_df, threshold_seconds=1):
#     import pandas as pd

#     grouped_fills_df = grouped_fills_df.copy()
#     trade_through_df = trade_through_df.copy()

#     # 確保 time 是欄位，不是 index（兩個 df 都做）
#     if grouped_fills_df.index.name == 'time' or 'time' not in grouped_fills_df.columns:
#         grouped_fills_df = grouped_fills_df.reset_index()

#     if trade_through_df.index.name == 'time' or 'time' not in trade_through_df.columns:
#         trade_through_df = trade_through_df.reset_index()

#     grouped_fills_df = grouped_fills_df.sort_values("time")
#     trade_through_df = trade_through_df.sort_values("time")

#     # 確保只有必要欄位，不要有衝突
#     if 'trade_through' not in trade_through_df.columns:
#         raise ValueError("trade_through_df must contain 'trade_through' column")

#     trade_through_df = trade_through_df[["time", "trade_through"]]

#     # merge_asof
#     merged = pd.merge_asof(
#         grouped_fills_df,
#         trade_through_df,
#         on="time",
#         direction="backward",
#         tolerance=pd.Timedelta(seconds=threshold_seconds)
#     )

#     merged["is_trade_near_cross"] = merged["trade_through"].notna()
#     return merged



In [ ]:
# grouped_fills = group_fills_within_seconds(fills_df, threshold_seconds=1)
# merged_sec_df = match_grouped_fills_to_trade_through(grouped_fills, trade_through_df, threshold_seconds=1)


In [45]:
# analyze_ratio_count_and_volume(merged_sec_df)

### Neutrino

In [60]:
warnings.filterwarnings("ignore", category=DeprecationWarning)
start_date = '2025-04-25'
end_date = '2025-05-06'
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

tmf_books, tmf_ticks = pd.DataFrame(), pd.DataFrame()

for date in tqdm(date_range, desc="Loading TMF data by day"):
    try:
        tmf_book = cmd.book_printer_v2(
            'FUT_TAIFEX_TMF:202505',
            sdate=date,
            edate=date,
            depth=5,
            source='SRC_TAIFEX_RT_RD.ORIG_CPBQ.DEST_CPBQ'
        )
        tmf_tick = cmd.tick_printer(
            'FUT_TAIFEX_TMF:202505',
            sdate=date,
            edate=date
        )

        tmf_books = pd.concat([tmf_books, tmf_book], axis=0)
        tmf_ticks = pd.concat([tmf_ticks, tmf_tick], axis=0)

    except Exception:
        continue


Loading TMF data by day:  17%|█▋        | 2/12 [00:45<03:25, 20.53s/it]WARNING: MYSQL_OPT_RECONNECT is deprecated and will be removed in a future version.
missing datafile SRC_TAIFEX.ORIG_CPBQ.DEST_CPBQ FUT_TAIFEX_TMF 20250427 file=/nfs/datafiles/2025/04/27/60225/datafile.944111726.gz
Loading TMF data by day:  75%|███████▌  | 9/12 [02:50<00:51, 17.12s/it]WARNING: MYSQL_OPT_RECONNECT is deprecated and will be removed in a future version.
missing datafile SRC_TAIFEX.ORIG_CPBQ.DEST_CPBQ FUT_TAIFEX_TMF 20250504 file=/nfs/datafiles/2025/05/04/60225/datafile.944111726.gz
Loading TMF data by day: 100%|██████████| 12/12 [04:10<00:00, 20.84s/it]


In [79]:
tmf_ticks_for_analysis = tmf_ticks.copy()
tmf_ticks_for_analysis['ex_time'] = pd.to_datetime(tmf_ticks_for_analysis['ex_time'])
tmf_ticks_for_analysis = tmf_ticks_for_analysis.rename(columns={'ex_time': 'time'}).set_index('time')

In [80]:
merged_df = pd.merge(tmf_ticks_for_analysis[['sz','px']], tmf_books, on='time', how='outer', sort=True)
merged_df['sz'] = merged_df['sz'].fillna(0)

merged_df[tmf_book.columns] = merged_df[tmf_book.columns].ffill()

In [81]:
traded_tmf = merged_df[merged_df['sz'] != 0].copy()

# 定義穿價條件（避免 ask_px4 或 bid_px4 為 0）
up_cross_tmf = (traded_tmf['ask_px2'] != 0) & (traded_tmf['px'] > traded_tmf['ask_px2'])
down_cross_tmf = (traded_tmf['bid_px2'] != 0) & (traded_tmf['px'] < traded_tmf['bid_px2'])

# 建立穿價標籤
traded_tmf['trade_through'] = 'none'
traded_tmf.loc[up_cross_tmf, 'trade_through'] = 'upward'
traded_tmf.loc[down_cross_tmf, 'trade_through'] = 'downward'


# 只留下有穿價的成交
tmf_trade_through_df = traded_tmf[traded_tmf['trade_through'] != 'none']

In [16]:
txf = RedisFillLoader('2025,4,25','2025,5,8','capital_neutrino_txf')

In [17]:
df2 = txf.load_concat_df()
fills_df2 = df2[df2['instr'].str.contains('txf:202505', case=False, na=False)]

Concatenating fills: 100%|██████████| 14/14 [00:00<00:00, 17.41it/s]


In [18]:
merged_fills2 = match_fills_to_trade_through(fills_df2, trade_through_df, threshold_seconds=0.01)

In [37]:
import numpy as np

def analyze_ratio_count_and_volume2(df):
    df = df.copy()
    df['trade_through'] = df['trade_through'].fillna('no-through')

    for label, sub_df in [('long', df[df['sz'] > 0]), ('short', df[df['sz'] < 0])]:
        if sub_df.empty:
            print(f"\n沒有 {label.upper()} TMF")
            continue

        # 成交量加權機率
        weights = sub_df['sz'].abs()

        # 先轉成 one-hot 類型欄位
        upward = (sub_df['trade_through'] == 'upward').astype(int)
        downward = (sub_df['trade_through'] == 'downward').astype(int)
        no_through = (sub_df['trade_through'] == 'no-through').astype(int)

        # 加權平均
        weighted_upward = np.average(upward, weights=weights)
        weighted_downward = np.average(downward, weights=weights)
        weighted_no_through = np.average(no_through, weights=weights)

        
        ratio_count = sub_df['trade_through'].value_counts(normalize=True)

        print(f"Neutrino {label.upper()} TXF:202505 時:")
        print(f"成交口數加權機率 -> 上穿: {weighted_upward:.2%}, 下穿: {weighted_downward:.2%}, 無穿價: {weighted_no_through:.2%}")


In [40]:
analyze_ratio_count_and_volume2(merged_fills2[merged_fills['alqortake'] != 'take'])

Neutrino LONG TXF:202505 時:
成交口數加權機率 -> 上穿: 1.72%, 下穿: 60.34%, 無穿價: 37.93%
Neutrino SHORT TXF:202505 時:
成交口數加權機率 -> 上穿: 44.23%, 下穿: 5.77%, 無穿價: 50.00%


In [24]:
analyze_ratio_count_and_volume(merged_fills[merged_fills['alqortake'] == 'take'])


Electron LONG 202505TMF 時 :
使用成交口數加權-> TXF 價格上穿: 23.74%, TXF 價格下穿: 6.08%, 無穿價: 70.18%

Electron SHORT 202505TMF 時 :
使用成交口數加權-> TXF 價格上穿: 2.18%, TXF 價格下穿: 61.13%, 無穿價: 36.69%
